In [1]:
import numpy as np
import pandas as pd
import tifffile
import napari
from scipy import ndimage as ndi
from skimage.filters import threshold_otsu,gaussian
from skimage.morphology import ball, binary_dilation, binary_opening
from skimage import exposure
from skimage.exposure import rescale_intensity
from skimage.measure import label, regionprops
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from sklearn.metrics import pairwise_distances
import json
import zarr
from ast import literal_eval
pd.options.mode.chained_assignment = None

### Helper functions

In [2]:
def erase_small_in_z(labels, min_size):
    rps = regionprops(labels)
    for r in rps:
        len_in_z = r.bbox[3]-r.bbox[0]
        if len_in_z<min_size:
            labels[labels==r.label]=0
    return labels

In [3]:
def erase_small(labels, min_size):
    unique, counts = np.unique(labels, return_counts=True)
    for i in range(len(unique)):
        if counts[i]<min_size:
            labels[labels==unique[i]]=0
    return labels

In [4]:
def remove_backg(img, sig1=1,sig2=15):
    img_sig =ndi.gaussian_filter(img, (sig1, sig1,sig1))
    img_bcg = ndi.gaussian_filter(img, (sig2,sig2,sig2))
    cleaned = img_sig - img_bcg
    cleaned[cleaned<0]=0
    return cleaned

In [5]:
def edistance (loc1,loc2):
    return np.sqrt((loc1[0]-loc2[0])**2 + (loc1[1]-loc2[1])**2 + (loc1[2]-loc2[2])**2)

In [6]:
def cost(location1, location2, matching_threshold):
    max_cost = 2 * matching_threshold

    # First check of the nodes are part of the same segment
    pre_dist = edistance(location1, location2)
    if pre_dist > matching_threshold:
        return max_cost
    dist = pre_dist
    return dist

In [7]:
def cost_matrix(predicted, immuno, matching_threshold):
    print("Computing matching costs...")

    size = max(len(predicted), len(immuno))
    costs = np.zeros((size, size), dtype=np.float32)
    costs[:] = 2 * matching_threshold
    num_potential_matches = 0
    for i in range(len(predicted)):
        for j in range(len(immuno)):
            c = cost(predicted[i], immuno[j], matching_threshold)
            costs[i, j] = c
            if c <= matching_threshold:
                num_potential_matches += 1

    print(str(num_potential_matches) + " potential matches found")

    return costs

In [8]:
def compute_molecule_detection_quality(locs_gt, locs_detected, matching_threshold):
    costs = cost_matrix(locs_detected, locs_gt, matching_threshold)
    matches = linear_sum_assignment(costs - np.amax(costs) - 1)
    matches = zip(matches[0], matches[1])  # scipy returns matches as numpy arrays

    filtered_matches = [(i, j) for (i, j) in matches if costs[i][j] <= matching_threshold]
    fp = len(locs_detected) - len(filtered_matches)

    # unmatched in gt = FN
    fn = len(locs_gt) - len(filtered_matches)
    
        # all ground truth elements - FN = TP
    tp = len(locs_gt) - fn
    
    precision = float(tp) / (tp + fp) if (tp + fp) > 0 else 0
    recall = float(tp) / (tp + fn) if (tp + fn) > 0 else 0
    if (precision + recall) > 0:
        fscore = 2.0 * precision * recall / (precision + recall)
    else:
        fscore = 0.0
    print("TP:" + str(tp))
    print("FP:" + str(fp))
    print("FN:" + str(fn))
    print('Precision:' + str(precision))
    print('Recall:' + str(recall))
    print('F1 score:' + str(fscore))
    return precision,recall,fscore,filtered_matches

In [9]:
def generate_matching_map(presyn_coords,postsyn_coords,matching_threshold=30):
    locs_pre_synapse = np.stack(presyn_coords)
    locs_post_synapse = np.stack(postsyn_coords)
    costs = cost_matrix(locs_post_synapse, locs_pre_synapse,matching_threshold)

    matches = linear_sum_assignment(costs - np.amax(costs) - 1)
    matches = zip(matches[0], matches[1])  # scipy returns matches as numpy arrays

    filtered_matches = [(i, j) for (i, j) in matches if costs[i][j] <= matching_threshold]
    matches_map = np.zeros(costs.shape)

    for i in range(costs.shape[0]):
        post_pre_connection_id = np.argmin(costs[i,:])
        if np.amin(costs[i,:])<=matching_threshold:
            matches_map[i,post_pre_connection_id]=1

    for i in range(costs.shape[0]):
        pre_post_connection_id = np.argmin(costs[:,i])
        if (np.amin(costs[:,i])<=matching_threshold) and (sum(matches_map[:,i])==0):
            matches_map[pre_post_connection_id ,i]=1
    return matches_map,filtered_matches

In [10]:
def match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon,filtered_matches_shank2):

    inv_filtered_matches_shank2 = dict((v, k) for k, v in dict(filtered_matches_shank2).items())
    inv_filtered_matches_bassoon = dict((v, k) for k, v in dict(filtered_matches_bassoon).items())

    full_synapses_gt_and_detected = full_synapses_gt
    full_synapses_gt_and_detected['post_id_detected_matched'] = -1
    full_synapses_gt_and_detected['pre_id_detected_matched'] = -1

    for i in range(len(full_synapses_gt_and_detected)):
        post_id = full_synapses_gt_and_detected['post_id'].loc[i]
        ind = post_synapses_gt[post_synapses_gt['id']==post_id].index[0]
        if ind in dict(inv_filtered_matches_shank2):
            full_synapses_gt_and_detected['post_id_detected_matched'].loc[i] = dict(inv_filtered_matches_shank2)[ind]

    for i in range(len(full_synapses_gt_and_detected)):
        pre_id = full_synapses_gt_and_detected['pre_id'].loc[i]
        ind = pre_synapses_gt[pre_synapses_gt['id']==pre_id].index[0]
        if ind in dict(inv_filtered_matches_bassoon):
            full_synapses_gt_and_detected['pre_id_detected_matched'].loc[i] = dict(inv_filtered_matches_bassoon)[ind]

    return full_synapses_gt_and_detected

In [11]:
def compute_full_synapse_detection_quality(full_synapses_gt_and_detected,matches_map):
#add column that stores information about the detection
    full_synapses_gt_and_detected['Detection is correct'] = 'No'

    #check if the deected pre-and-post are connected inn the detection as well
    for i in range(len(full_synapses_gt_and_detected)):
        post_id = full_synapses_gt_and_detected['post_id_detected_matched'].loc[i]
        pre_id = full_synapses_gt_and_detected['pre_id_detected_matched'].loc[i]
        if matches_map[post_id,pre_id]>0:
            full_synapses_gt_and_detected['Detection is correct'].loc[i]='Yes'
            matches_map[post_id,pre_id]=2

    un, counts = np.unique(matches_map, return_counts = True)
    fp = counts[1]
    print('FP full-synapse:' + str(fp))
    fp_ids = np.where(matches_map==1)
    fp_ids_post = fp_ids[0]
    fp_ids_pre = fp_ids[1]

    tps = full_synapses_gt_and_detected[full_synapses_gt_and_detected["Detection is correct"]=='Yes']
    tp = len(tps)
    print('TP full-synapse:' + str(tp))

    fns = full_synapses_gt_and_detected[full_synapses_gt_and_detected["Detection is correct"]=='No']
    fn = len(fns)
    print('FN full-synapse:' + str(fn))
   
    precision = float(tp) / (tp + fp) if (tp + fp) > 0 else 0
    recall = float(tp) / (tp + fn) if (tp + fn) > 0 else 0
    if (precision + recall) > 0:
        fscore = 2.0 * precision * recall / (precision + recall)
    else:
        fscore = 0.0

    print('Precision:' + str(precision))
    print('Recall:' + str(recall))
    print('F1 score:' + str(fscore))
    return precision,recall,fscore

### 1. Load imaging data

#### 1.1 Load gt data

In [12]:
img_path = '../../Data/LICONN_test_dataset.tif'
gt_image = tifffile.imread(img_path)

In [13]:
bassoon = gt_image[:,1,:,:]
shank2 = gt_image[:,0,:,:]
liconn = gt_image[:,2,:,:]

In [14]:
viewer = napari.Viewer()
viewer.add_image(liconn,colormap='gray')
viewer.add_image(bassoon,colormap = 'cyan', blending = 'additive')
viewer.add_image(shank2,colormap = 'magenta', blending = 'additive')

<Image layer 'shank2' at 0x1fbc09e4040>

#### 1.2 Load predicted signal

In [27]:
file_name = "../../Data/bassoon_predcition.zarr"
container = zarr.open(file_name)
data = container["prediction"]
bassoon_predicted = data[0,59:259,15:715,49:749] #prediction was done on the full tile, crop it to the size of the test ROI

In [28]:
file_name = "../../Data/shank2_prediction.zarr"
container = zarr.open(file_name)
data = container["prediction"]
shank2_predicted = data[0,59:259,15:715,49:749] #prediction was done on the full tile, crop it to the size of the test ROI

### 2. Load GT annotations

In [16]:
# Opening JSON file
f = open('../../Data/LICONN_test_dataset_full_synapses.json')
data = json.load(f)

In [17]:
pre_synapses_gt = pd.DataFrame(columns = ['id', 'location'])
post_synapses_gt = pd.DataFrame(columns = ['id', 'location'])
full_synapses_gt = pd.DataFrame(columns = ['post_id', 'post_coordinate', 'pre_id', 'pre_coordinate'])

In [18]:
for i in range(len(data['partners'])):
    
    pre_id = int(data['partners'][i]['pre-synapse'])
    pre_location = data['ids'][str(pre_id)]['location']
    data_pre = [[pre_id, pre_location[::-1]]]
    pre_synapses_gt = pd.concat([pre_synapses_gt,pd.DataFrame(columns = ['id', 'location'], data = data_pre)],ignore_index=True)
    
    post_id = int(data['partners'][i]['post-synapse'])
    post_location = data['ids'][str(post_id)]['location']
    data_post = [[post_id, post_location[::-1]]]
    post_synapses_gt = pd.concat([post_synapses_gt,pd.DataFrame(columns = ['id', 'location'], data = data_post)],ignore_index=True)
    
    full_data = [[post_id, post_location[::-1], pre_id, pre_location[::-1]]]
    full_synapses_gt =pd.concat([full_synapses_gt,pd.DataFrame(columns = ['post_id', 'post_coordinate', 'pre_id', 'pre_coordinate'], data = full_data)], ignore_index=True)

In [19]:
pre_synapses_gt = pre_synapses_gt.drop_duplicates(subset=["id"], keep='first')
post_synapses_gt = post_synapses_gt.drop_duplicates(subset=["id"], keep='first')

In [20]:
f = open('../../Data/LICONN_test_dataset_single_pre.json')
data = json.load(f)

In [21]:
pre_synapses_gt_single = pd.DataFrame(columns = ['id', 'location'])
post_synapses_gt_single = pd.DataFrame(columns = ['id', 'location'])

In [22]:
for i in list(data['ids']):
    pre_id = int(i)
    pre_location = data['ids'][str(pre_id)]['location']
    data_pre = [[pre_id, pre_location[::-1]]]
    pre_synapses_gt_single = pd.concat([pre_synapses_gt_single,pd.DataFrame(columns = ['id', 'location'], data = data_pre)],ignore_index=True)

In [23]:
f = open('../../Data/LICONN_test_dataset_single_post.json')
data = json.load(f)

In [24]:
for i in list(data['ids']):
    post_id = int(i)
    post_location = data['ids'][str(post_id)]['location']
    data_post = [[post_id, post_location[::-1]]]
    post_synapses_gt_single = pd.concat([post_synapses_gt_single,pd.DataFrame(columns = ['id', 'location'], data = data_post)],ignore_index=True)

In [25]:
pre_synapses_gt = pd.concat([pre_synapses_gt,pre_synapses_gt_single],ignore_index=True)
post_synapses_gt = pd.concat([post_synapses_gt,post_synapses_gt_single],ignore_index=True)

In [26]:
locs_bassoon_gt = np.stack(pre_synapses_gt['location'].values)
locs_shank2_gt = np.stack(post_synapses_gt['location'].values)

### 3. Detect synases from predicted signal

##### 3.1 bassoon (pre-synapse) detection

In [29]:
bassoon_predicted_rescaled = rescale_intensity(bassoon_predicted, in_range=(np.percentile(bassoon_predicted,1),  np.percentile(bassoon_predicted, 99.95)))
bassoon_predicted_rescaled_smoothed = ndi.gaussian_filter(bassoon_predicted_rescaled, (5,5,5))

thr = threshold_otsu(bassoon_predicted_rescaled_smoothed)
bassoon_prediction_mask = bassoon_predicted_rescaled_smoothed>thr

In [30]:
labels_bassoon_predicted = label(bassoon_prediction_mask)
labels_bassoon_predicted = erase_small(labels_bassoon_predicted, 60)
rp = regionprops(labels_bassoon_predicted, intensity_image = liconn)

# save not z-filtered results for the final post-processing step
presyn_coords_predicted_nozclean = []
for r in rp:
    presyn_coords_predicted_nozclean.append(np.array(r.centroid_weighted))
    
labels_bassoon_predicted  = erase_small_in_z(labels_bassoon_predicted, 13)
rp = regionprops(labels_bassoon_predicted, intensity_image = liconn)
presyn_coords_predicted = []
for r in rp:
    presyn_coords_predicted.append(np.array(r.centroid_weighted))

##### 3.2 shank2 (post-synapse) detection

In [31]:
shank2_predicted_rescaled = rescale_intensity(shank2_predicted, in_range=(np.percentile(shank2_predicted,1),  np.percentile(shank2_predicted, 99.95)))
shank2_predicted_rescaled_smoothed = ndi.gaussian_filter(shank2_predicted_rescaled, (5,5,5))

thr = threshold_otsu(shank2_predicted_rescaled_smoothed)
shank2_prediction_mask = shank2_predicted_rescaled_smoothed>thr

In [32]:
labels_shank2_predicted = label(shank2_prediction_mask)
labels_shank2_predicted = erase_small(labels_shank2_predicted, 60)
rp = regionprops(labels_shank2_predicted, intensity_image = liconn)

# save not z-filtered results for the final post-processing step
postsyn_coords_predicted_nozclean = []
for r in rp:
    postsyn_coords_predicted_nozclean.append(np.array(r.centroid_weighted))
    
labels_shank2_predicted = erase_small_in_z(labels_shank2_predicted, 13)
rp = regionprops(labels_shank2_predicted, intensity_image = liconn)
postsyn_coords_predicted = []
for r in rp:
    postsyn_coords_predicted.append(np.array(r.centroid_weighted))

##### 3.3 match pre and post synapses to detect full synapses (generate matches map)

In [35]:
matches_map_initial,filtered_matches_predicted = generate_matching_map(presyn_coords_predicted,postsyn_coords_predicted,matching_threshold=30)

Computing matching costs...
215 potential matches found


### 4. Validation

##### 4.1 Single bassoon and shank2 detection validation

In [36]:
matching_threshold = 30

In [37]:
precision,recall,fscore,filtered_matches_bassoon = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_predicted), matching_threshold)

Computing matching costs...
249 potential matches found
TP:227
FP:8
FN:34
Precision:0.9659574468085106
Recall:0.8697318007662835
F1 score:0.9153225806451613


In [38]:
precision,recall,fscore,filtered_matches_shank2 = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_predicted), matching_threshold)

Computing matching costs...
238 potential matches found
TP:213
FP:0
FN:34
Precision:1.0
Recall:0.8623481781376519
F1 score:0.9260869565217391


##### 4.2 Full synapse detection quality

In [41]:
#match detected bassoon and shank2 with full gt synapses
full_synapses_gt_and_predicted = match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon,filtered_matches_shank2)

In [42]:
#compute F1 score of full synapse detection
fprecision,frecall,ffscore = compute_full_synapse_detection_quality(full_synapses_gt_and_predicted,matches_map_initial)

FP full-synapse:7
TP full-synapse:202
FN full-synapse:40
Precision:0.9665071770334929
Recall:0.8347107438016529
F1 score:0.8957871396895787


### Post-processing

##### Second look proofreading step

In [43]:
def get_new_detections(bbox,gt_image,labels_bassoon,labels_shank2,current_presyn_coordinate, matching_threshold):
    z0,y0,x0,z1,y1,x1 = bbox
    bbassoon = gt_image[z0:z1,1,y0:y1,x0:x1]
    bshank2 = gt_image[z0:z1,0,y0:y1,x0:x1]
    bliconn= gt_image[z0:z1,2,y0:y1,x0:x1]
    mi = np.percentile(bliconn, 1)
    ma = np.percentile(bliconn, 99.95)
    img_rescale = rescale_intensity(bliconn, in_range=(mi, ma))
    cleaned_pan = remove_backg(img_rescale.astype('float32'), sig1=2,sig2=10)
    mi = np.percentile(cleaned_pan, 95)
    ma = np.percentile(cleaned_pan, 100)
    threshold = 0.4*(ma-mi) + mi
    pan_mask = cleaned_pan>threshold
    mask_existing_labels = labels_shank2[z0:z1,y0:y1,x0:x1] + labels_bassoon[z0:z1,y0:y1,x0:x1]
    mask_existing_labels = mask_existing_labels>0
    potential_shank2_mask = pan_mask*~mask_existing_labels
    potential_shank2_mask = binary_opening(potential_shank2_mask, ball(2))
    labels_potential_shank2 = label(potential_shank2_mask)
    labels_potential_shank2 = erase_small(labels_potential_shank2, 20)
    rp = regionprops(labels_potential_shank2, intensity_image = bliconn)
    new_postsyn_detections = []
    for r in rp:
        new_postsyn_detections.append(np.array(r.centroid_weighted))
    final_new_shank2_list = []
    for i in range(len(new_postsyn_detections)):
        shank2_pos = new_postsyn_detections[i] + [z0,y0,x0]
        bassoon_pos = current_presyn_coordinate
        if edistance (shank2_pos,bassoon_pos )<matching_threshold:
            final_new_shank2_list.append(shank2_pos)  
    return final_new_shank2_list

In [44]:
doublecheck_detection_bassoon_list = []
#find all bassoons without a pair
for i in range(len(matches_map_initial)):
    if sum(matches_map_initial[:,i])==0: #and i not in bassoon_detected_ids_to_remove:
        doublecheck_detection_bassoon_list.append(i)

In [45]:
doublecheck_detection_shank2_list = []
#find all shank2s without a pair
for i in range(len(matches_map_initial)):
    if sum(matches_map_initial[i,:])==0: #and i not in shank2_detected_ids_to_remove:
        doublecheck_detection_shank2_list.append(i)

In [46]:
search_dist = 50
i=0
all_new_shank2 = []
for ind in doublecheck_detection_bassoon_list:
    coordinate = presyn_coords_predicted[ind].astype('uint16')
    z0 = max(0,coordinate[0]-search_dist)
    z1 = min(coordinate[0] + search_dist, bassoon.shape[0])
    z_mid = ((z1-z0)/2).astype('uint16')
    y0 = max(0,coordinate[1]-search_dist)
    y1 = min(coordinate[1] + search_dist, bassoon.shape[1])
    x0 = max(0,coordinate[2]-search_dist)
    x1 = min(coordinate[2] + search_dist, bassoon.shape[2])
    final_new_shank2_list = get_new_detections([z0,y0,x0,z1,y1,x1],gt_image,labels_bassoon_predicted,labels_shank2_predicted,coordinate, matching_threshold)
    if len(final_new_shank2_list)!=0:
        for j in range(len(final_new_shank2_list)):
            all_new_shank2.append(final_new_shank2_list[j])

In [47]:
postsyn_coords_predicted_after_doublecheck = postsyn_coords_predicted+all_new_shank2

In [48]:
precision,recall,fscore,filtered_matches_shank2 = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_predicted_after_doublecheck), matching_threshold)

Computing matching costs...
248 potential matches found
TP:221
FP:5
FN:26
Precision:0.9778761061946902
Recall:0.8947368421052632
F1 score:0.9344608879492601


In [49]:
search_dist = 50
all_new_bassoons = []
for ind in doublecheck_detection_shank2_list:
    if ind < len(postsyn_coords_predicted):
        coordinate = postsyn_coords_predicted[ind].astype('uint16')
        z0 = max(0,coordinate[0]-search_dist)
        z1 = min(coordinate[0] + search_dist, bassoon.shape[0])
        z_mid = ((z1-z0)/2).astype('uint16')
        y0 = max(0,coordinate[1]-search_dist)
        y1 = min(coordinate[1] + search_dist, bassoon.shape[1])
        x0 = max(0,coordinate[2]-search_dist)
        x1 = min(coordinate[2] + search_dist, bassoon.shape[2])
        final_new_bassoon_list = get_new_detections([z0,y0,x0,z1,y1,x1],gt_image,labels_bassoon_predicted,labels_shank2_predicted,coordinate, matching_threshold)
        if len(final_new_bassoon_list)!=0:
            for j in range(len(final_new_bassoon_list)):
                all_new_bassoons.append(final_new_bassoon_list[j])

In [50]:
presyn_coords_predicted_after_doublecheck = presyn_coords_predicted+all_new_bassoons

In [51]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_predicted_after_doublecheck), matching_threshold)

Computing matching costs...
249 potential matches found
TP:227
FP:8
FN:34
Precision:0.9659574468085106
Recall:0.8697318007662835
F1 score:0.9153225806451613


In [52]:
matches_map_doublecheck,  filtered_matches_predicted =  generate_matching_map(presyn_coords_predicted_after_doublecheck,postsyn_coords_predicted_after_doublecheck,matching_threshold=30)

Computing matching costs...
228 potential matches found


In [55]:
full_synapses_gt_and_predicted_after_doublecheck = match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon,filtered_matches_shank2)

In [56]:
fprecision,frecall,ffscore = compute_full_synapse_detection_quality(full_synapses_gt_and_predicted_after_doublecheck,matches_map_doublecheck)

FP full-synapse:13
TP full-synapse:209
FN full-synapse:33
Precision:0.9414414414414415
Recall:0.8636363636363636
F1 score:0.9008620689655172


##### Border cleaning step

In [57]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon_nozfilter = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_predicted_nozclean), matching_threshold)

Computing matching costs...
264 potential matches found
TP:241
FP:20
FN:20
Precision:0.9233716475095786
Recall:0.9233716475095786
F1 score:0.9233716475095786


In [58]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon_init = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_predicted_after_doublecheck), matching_threshold)

Computing matching costs...
249 potential matches found
TP:227
FP:8
FN:34
Precision:0.9659574468085106
Recall:0.8697318007662835
F1 score:0.9153225806451613


In [59]:
tp_ids_init = list(zip(*filtered_matches_bassoon_init))[1]
tp_ids_nozfilter = list(zip(*filtered_matches_bassoon_nozfilter))[1]
s = set(tp_ids_init)
potential_tps = [x for x in tp_ids_nozfilter if x not in s]

In [60]:
inv_filtered_matches_bassoon_nozfilter = dict((v, k) for k, v in dict(filtered_matches_bassoon_nozfilter).items())

In [61]:
tps_to_keep = []
new_presyn_predicted = []
new_tps=0
for i in potential_tps:
    if locs_bassoon_gt[i][0]<=10 or  locs_bassoon_gt[i][0]>=190:
        tps_to_keep.append(i)
        new_tps+=1
        new_presyn_predicted.append(presyn_coords_predicted_nozclean[inv_filtered_matches_bassoon_nozfilter[i]])
        #here we append the detections matched with these GTs backt o the presyn_coords_immuno list

In [62]:
presyn_coords_predicted_after_doublecheck_borders = new_presyn_predicted + presyn_coords_predicted_after_doublecheck

In [63]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_shank2_nozfilter = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_predicted_nozclean), matching_threshold)

Computing matching costs...
259 potential matches found
TP:230
FP:4
FN:17
Precision:0.9829059829059829
Recall:0.9311740890688259
F1 score:0.9563409563409564


In [64]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_shank2_init = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_predicted_after_doublecheck), matching_threshold)

Computing matching costs...
248 potential matches found
TP:221
FP:5
FN:26
Precision:0.9778761061946902
Recall:0.8947368421052632
F1 score:0.9344608879492601


In [65]:
tp_ids_init = list(zip(*filtered_matches_shank2_init))[1]
tp_ids_nozfilter = list(zip(*filtered_matches_shank2_nozfilter))[1]
s = set(tp_ids_init)
potential_tps = [x for x in tp_ids_nozfilter if x not in s]

In [66]:
inv_filtered_matches_shank2_nozfilter = dict((v, k) for k, v in dict(filtered_matches_shank2_nozfilter).items())

In [67]:
tps_to_keep = []
new_postsyn_predicted = []
new_tps=0
for i in potential_tps:
    if locs_shank2_gt[i][0]<=10 or  locs_shank2_gt[i][0]>=190:
        tps_to_keep.append(i)
        new_tps+=1
        new_postsyn_predicted.append(postsyn_coords_predicted_nozclean[inv_filtered_matches_shank2_nozfilter[i]])
        #here we append the detections matched with these GTs backt o the presyn_coords_immuno list

In [68]:
postsyn_coords_predicted_after_doublecheck_borders = new_postsyn_predicted + postsyn_coords_predicted_after_doublecheck

In [69]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon_final = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_predicted_after_doublecheck_borders), matching_threshold)

Computing matching costs...
261 potential matches found
TP:238
FP:8
FN:23
Precision:0.967479674796748
Recall:0.9118773946360154
F1 score:0.9388560157790928


In [70]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_shank2_final = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_predicted_after_doublecheck_borders), matching_threshold)

Computing matching costs...
258 potential matches found
TP:230
FP:5
FN:17
Precision:0.9787234042553191
Recall:0.9311740890688259
F1 score:0.954356846473029


In [71]:
matches_map_doublecheck_final, filtered_matches_immuno_detecion_final = generate_matching_map(presyn_coords_predicted_after_doublecheck_borders,postsyn_coords_predicted_after_doublecheck_borders,matching_threshold=30)

Computing matching costs...
237 potential matches found


In [72]:
#match detected bassoon and shank2 with full gt synapses
full_synapses_gt_and_predicted_after_doublecheck_final = match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon_final,filtered_matches_shank2_final)

In [73]:
#compute F1 score of full synapse detection
fprecision,frecall,ffscore = compute_full_synapse_detection_quality(full_synapses_gt_and_predicted_after_doublecheck_final,matches_map_doublecheck_final)

FP full-synapse:13
TP full-synapse:218
FN full-synapse:24
Precision:0.9437229437229437
Recall:0.9008264462809917
F1 score:0.9217758985200846
